**Exercise 2.** 
$\ minimize\  c^Tx = \sum_{i=1}^Nc_i x_i, \\ \text{s.t.}\  Ax \leq b \\ l\leq x\leq u \\ We\ will\ solve\ the\ general\ problem\ using\ concrete\ values\ given\ below, \\ Here\ N= 10 \\ c = [-9.6, -2.85, 41.5, 33.05, -0.75, 20.5, -0.35, -31.02, 50.65, 6.125];\\
A = \begin{bmatrix} 
0 & 1 & -2 & 0 & 1& -2 & 0 & 1 & 1& -1\\ 
-1 & 1 & -3 & 2.5 & 0 & 1.4 & 0 & -2 & 1 & -1.5 \\
-0.45 & 4.78 & -2 & -3.22 & -1.3 & 1 & -1 & 0 & 0 & 0.25 \\ 1 & 0 & 0\ & 0 & 1 & 1 & 0 &  2&  3 & 0
\end{bmatrix} \\ b = [35.4, 47.9, 41.43, 22.7]; \\ l = [-\infty, -\infty ,0 , 0, -67, 0, -23, -5, 6, 1] : \\ u = [4, \infty , +\infty , 2, 20, 55, 23, 15, +\infty, +\infty]$


In [38]:
!pip install -q pyomo

In [39]:
from pyomo.environ import *

In [40]:
import numpy as np

In [41]:
#create a model
model = ConcreteModel()

In [42]:
# we can set certain values using python variables
N = 10 #we declare a Python variable N denoting the number of decision variables  
M = 4 #we declare a Python variable M denoting the number of constraints 

In [43]:
# coefficients for objective function can be provided using numpy arrays
obj_coef_c = np.array([-9.6, -2.85, 41.5, 33.05, -0.75, 20.5, -0.35, -31.02, 50.65, 6.125])

In [44]:
# We use more numpy arrays for providing constraint coefficients
constr_coef_A = np.array([[0, 1, -2, 0, 1, -2, 0, 1, 1, -1] , [-1, 1, -3, 2.5, 0, 1.4, 0, -2, 1, -1.5], [-0.45, 4.78, -2, -3.22, -1.3, 1, -1, 0, 0, 0.25], [1, 0, 0, 0, 1, 1, 0, 2, 3, 0]])
print(constr_coef_A)

#declare the RHS of the constraints as variables
constr_rhs_b =np.array([35.4, 47.9, 41.43, 22.7])
print(constr_rhs_b)


[[ 0.    1.   -2.    0.    1.   -2.    0.    1.    1.   -1.  ]
 [-1.    1.   -3.    2.5   0.    1.4   0.   -2.    1.   -1.5 ]
 [-0.45  4.78 -2.   -3.22 -1.3   1.   -1.    0.    0.    0.25]
 [ 1.    0.    0.    0.    1.    1.    0.    2.    3.    0.  ]]
[35.4  47.9  41.43 22.7 ]


In [45]:
# set of column indices and row indices: since we have N variables, we can take column indices to be the list [0,1,2,...,N-1]
col_indices = np.arange(N)
print(col_indices)

row_indices = np.arange(M)
print(row_indices)


[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3]


In [46]:
#declare the decision variables in the model
model.x = Var(col_indices)

In [47]:
# create a ConstraintList to hold multiple constraints
model.constraints = ConstraintList()
for i in row_indices:
  model.constraints.add(sum(constr_coef_A[i,j]*model.x[j] for j in col_indices) <= constr_rhs_b[i])

In [48]:
#The following numpy arrays will be useful for setting the lower bound and upper bounds for the variables
lower_bound = np.array([-np.inf, -np.inf,0 , 0, -67, 0, -23, -5, 6, 1])
upper_bound = np.array([4, np.inf , np.inf, 2, 20, 55, 23, 15, np.inf, np.inf])

In [49]:
#add the upper bound and lower bounds for the variables
for j in col_indices:
  model.x[j].setlb(lower_bound[j])
  model.x[j].setub(upper_bound[j])

In [50]:
# add the model objective
model.objective = Objective(expr = sum(obj_coef_c[j]*model.x[j] for j in col_indices), sense=minimize)

In [51]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  -inf :  None :   4.0 : False :  True :  Reals
          1 :  -inf :  None :   inf : False :  True :  Reals
          2 :   0.0 :  None :   inf : False :  True :  Reals
          3 :   0.0 :  None :   2.0 : False :  True :  Reals
          4 : -67.0 :  None :  20.0 : False :  True :  Reals
          5 :   0.0 :  None :  55.0 : False :  True :  Reals
          6 : -23.0 :  None :  23.0 : False :  True :  Reals
          7 :  -5.0 :  None :  15.0 : False :  True :  Reals
          8 :   6.0 :  None :   inf : False :  True :  

In [52]:
!apt-get install -y -qq glpk-utils

In [53]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -196.378974895398
  Upper bound: -196.378974895398
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 11
  Number of nonzeros: 29
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.012407302856445312
# ----------------------------------------------------------
#   Solution Information
# ----------

In [55]:
print('\nObjective function value = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i+1,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()
print('2) Since the activity status of all the constraints is true which means that all of these factors causing the limitation on the objective function.')


Objective function value =  -196.3789748953975

Decision Variables
x[ 1 ] =  4.0
x[ 2 ] =  5.8347280334728
x[ 3 ] =  0.0
x[ 4 ] =  0.0
x[ 5 ] =  -29.3
x[ 6 ] =  0.0
x[ 7 ] =  23.0
x[ 8 ] =  15.0
x[ 9 ] =  6.0
x[ 10 ] =  1.0

Constraints
constraints : Size=4
    Key : Lower : Body               : Upper
      1 :  None : -3.465271966527201 :  35.4
      2 :  None :  -23.6652719665272 :  47.9
      3 :  None :  41.42999999999998 : 41.43
      4 :  None :               22.7 :  22.7
2) Since the activity status of all the constraints is true which means that all of these factors causing the limitation on the objective function.
